# Deep Dive into dbt (Data Build Tool)

## What is dbt?

**dbt (Data Build Tool)** is a transformation workflow tool that enables data analysts and engineers to transform data in their warehouse more effectively. It allows you to write modular SQL queries and build reliable data pipelines using software engineering best practices.

### Core Concepts

dbt sits at the **T** in ELT (Extract, Load, Transform):
- **Extract**: Data is extracted from source systems
- **Load**: Data is loaded into your data warehouse (MariaDB in our case)
- **Transform**: dbt transforms the raw data into analytics-ready models

### Why dbt?

Traditional data transformation has several problems:
1. **No version control**: SQL scripts scattered across different tools
2. **No testing**: Data quality issues discovered too late
3. **No documentation**: Tribal knowledge, hard to onboard new team members
4. **No dependency management**: Manual tracking of which tables depend on others
5. **No modularity**: Copy-paste SQL leading to maintenance nightmares

dbt solves these by:
- **Version controlling** all transformations in Git
- **Testing** data quality automatically
- **Documenting** models and generating documentation sites
- **Managing dependencies** automatically through ref() function
- **Enabling modularity** through reusable models and macros


## dbt Project Structure

After running `dbt init dbt_sql`, you get this structure:

```
DTB-FOR-SQL/
├── dbt_sql/                    # Main dbt project directory
│   ├── analyses/               # Analytical queries (SELECT only, not materialized)
│   ├── dbt_packages/           # External dbt packages (like Python pip packages)
│   ├── logs/                   # Execution logs
│   ├── macros/                 # Reusable SQL functions (Jinja macros)
│   ├── models/                 # WHERE THE MAGIC HAPPENS - Your transformation SQL
│   ├── seeds/                  # CSV files to load into database
│   ├── snapshots/              # Type 2 Slowly Changing Dimensions
│   ├── target/                 # Compiled SQL and artifacts (gitignored)
│   ├── tests/                  # Custom data tests
│   ├── .gitignore             # Git ignore file
│   ├── dbt_project.yml        # Project configuration
│   └── README.md              # Project documentation
├── logs/                       # Additional logs
├── venv/                       # Python virtual environment
├── .gitignore                 # Root gitignore
└── requirements.txt           # Python dependencies
```

### Key Directories Explained

#### 1. **models/**
This is where you write your transformation logic. Each `.sql` file becomes a table or view in your database.

Example structure:
```
models/
├── staging/           # Raw data cleaning and light transformation
│   └── stg_taxi_trips.sql
├── intermediate/      # Business logic transformations
│   └── int_taxi_metrics.sql
└── marts/            # Final analytics-ready tables
    └── fct_daily_revenue.sql
```

#### 2. **seeds/**
CSV files that get loaded into your database as tables. Perfect for:
- Reference data (country codes, category mappings)
- Small lookup tables
- Sample data for development

#### 3. **macros/**
Reusable SQL snippets using Jinja templating. Think of them as SQL functions.

#### 4. **tests/**
Custom data quality tests beyond dbt's built-in tests.

#### 5. **snapshots/**
Track historical changes to mutable tables (Type 2 SCDs).


## Setting Up the Project

### Step 1: Navigate to Project Directory

```bash
cd DTB-FOR-SQL
```

### Step 2: Activate Virtual Environment

```bash
# On Linux/Mac
source venv/bin/activate

# On Windows
venv\Scripts\activate
```

### Step 3: Install dbt with MariaDB Adapter

```bash
pip install dbt-mysql
```

Note: MariaDB uses the MySQL adapter since they're protocol-compatible.

### Step 4: Configure Database Connection

Edit `~/.dbt/profiles.yml`:

```yaml
dbt_sql:
  outputs:
    dev:
      type: mysql
      host: localhost
      port: 3306
      user: your_username
      password: your_password
      database: dbt_analytics
      schema: dbt_dev
      threads: 4
      
    prod:
      type: mysql
      host: localhost
      port: 3306
      user: prod_username
      password: prod_password
      database: dbt_analytics
      schema: dbt_prod
      threads: 8
      
  target: dev
```

### Step 5: Test Connection

```bash
dbt debug
```

You should see:
```
Connection test: [OK connection ok]
```


## Working with Seeds - Loading CSV Data

### What are Seeds?

Seeds are CSV files in your `dbt_sql/seeds/` directory that dbt can load into your database as tables. They're version-controlled and can be referenced in your models.

### Our Example: Taxi Trips Data

We have a file: `dbt_sql/seeds/raw_taxi_trips.csv`

Sample structure:
```csv
pickup,dropoff,passengers,distance,fare,tip,tolls,total,color,payment,pickup_zone,dropoff_zone,pickup_borough,dropoff_borough
2019-03-23 20:21:09,2019-03-23 20:27:24,1,1.6,7.0,2.15,0.0,12.95,yellow,credit card,Lenox Hill West,UN/Turtle Bay South,Manhattan,Manhattan
2019-03-04 16:11:55,2019-03-04 16:19:00,1,0.79,5.0,0.0,0.0,9.3,yellow,cash,Upper West Side South,Upper West Side South,Manhattan,Manhattan
2019-03-27 17:53:01,2019-03-27 18:00:25,1,1.37,7.5,2.36,0.0,14.16,yellow,credit card,Alphabet City,West Village,Manhattan,Manhattan
2019-03-10 01:23:59,2019-03-10 01:49:51,1,7.7,27.0,6.15,0.0,36.95,yellow,credit card,Hudson Sq,Yorkville West,Manhattan,Manhattan
2019-03-30 13:27:42,2019-03-30 13:37:14,3,2.16,9.0,1.1,0.0,13.4,yellow,credit card,Midtown East,Yorkville West,Manhattan,Manhattan
2019-03-11 10:37:23,2019-03-11 10:47:31,1,0.49,7.5,2.16,0.0,12.96,yellow,credit card,Times Sq/Theatre District,Midtown East,Manhattan,Manhattan
2019-03-26 21:07:31,2019-03-26 21:17:29,1,3.65,13.0,2.0,0.0,18.8,yellow,credit card,Battery Park City,Two Bridges/Seward Park,Manhattan,Manhattan
```


### Loading the Seed Data

```bash
# Navigate to dbt project directory
cd dbt_sql

# Load all seed files
dbt seed

# Load specific seed file
dbt seed --select raw_taxi_trips

# Full refresh (drop and recreate table)
dbt seed --full-refresh
```

### What Happens When You Run `dbt seed`?

1. **Reads CSV**: dbt reads `raw_taxi_trips.csv`
2. **Infers Schema**: Determines column types (VARCHAR, INT, DECIMAL, DATETIME)
3. **Creates Table**: Executes `CREATE TABLE` in MariaDB
4. **Loads Data**: Inserts all rows using batch inserts
5. **Logs Results**: Shows success/failure in console

The table created: `dbt_analytics.dbt_dev.raw_taxi_trips`


## Viewing Seed Data - Three Methods

Let's verify our seed data loaded correctly using three different approaches:
1. MariaDB CLI
2. Python with pandas
3. dbt commands


### Method 1: MariaDB Command Line

```bash
# Connect to MariaDB
mysql -u your_username -p

# Or connect directly to database
mysql -u your_username -p dbt_analytics
```

Once connected:

```sql
-- Switch to database
USE dbt_analytics;

-- Show all tables
SHOW TABLES;

-- Describe table structure
DESCRIBE raw_taxi_trips;

-- View first 10 rows
SELECT * FROM raw_taxi_trips LIMIT 10;

-- Count total rows
SELECT COUNT(*) as total_trips FROM raw_taxi_trips;

-- Basic analytics
SELECT 
    DATE(pickup_datetime) as trip_date,
    COUNT(*) as num_trips,
    ROUND(AVG(trip_distance), 2) as avg_distance,
    ROUND(AVG(fare_amount), 2) as avg_fare,
    ROUND(SUM(total_amount), 2) as total_revenue
FROM raw_taxi_trips
GROUP BY DATE(pickup_datetime)
ORDER BY trip_date;

-- Exit
EXIT;
```


### Method 2: Python with pandas and SQLAlchemy

Let's create a Python script to query the seeded data:


In [1]:
# Install required packages first
# pip install pandas sqlalchemy pymysql

import pandas as pd
from sqlalchemy import create_engine, text
import warnings
warnings.filterwarnings('ignore')

# Load environment variables (if any)
import os
from dotenv import load_dotenv
load_dotenv()


True

In [4]:
USER = os.getenv('DB_USER', 'rellika')
PASSWORD = os.getenv('DB_PASSWORD', 'krellika')
HOST = os.getenv('DB_HOST', 'localhost')
PORT = os.getenv('DB_PORT', '3306')
DATABASE = os.getenv('DB_NAME', 'dbt_sql')

# Database connection parameters
DB_CONFIG = {
    'user': USER,
    'password': PASSWORD,
    'host': HOST,
    'port': int(PORT),
    'database': DATABASE
}

# Create connection string
connection_string = f"mysql+pymysql://{DB_CONFIG['user']}:{DB_CONFIG['password']}@{DB_CONFIG['host']}:{DB_CONFIG['port']}/{DB_CONFIG['database']}"

# Create engine
engine = create_engine(connection_string)

print("Connection established successfully!")

Connection established successfully!


In [6]:
# Read the entire seed table
query = "SELECT * FROM raw_taxi_trips"
df_taxi = pd.read_sql(query, engine)

print(f"Total rows loaded: {len(df_taxi)}")
df_taxi.head()

Total rows loaded: 1999


,pickup,dropoff,passengers,distance,fare,tip,tolls,total,color,payment,pickup_zone,dropoff_zone,pickup_borough,dropoff_borough
0,2019-03-23 20:21:09,2019-03-23 20:27:24,1,1.60,7.0,2.15,0.0,12.95,yellow,credit card,Lenox Hill West,UN/Turtle Bay South,Manhattan,Manhattan
1,2019-03-04 16:11:55,2019-03-04 16:19:00,1,0.79,5.0,0.00,0.0,9.30,yellow,cash,Upper West Side South,Upper West Side South,Manhattan,Manhattan
2,2019-03-27 17:53:01,2019-03-27 18:00:25,1,1.37,7.5,2.36,0.0,14.16,yellow,credit card,Alphabet City,West Village,Manhattan,Manhattan
3,2019-03-10 01:23:59,2019-03-10 01:49:51,1,7.70,27.0,6.15,0.0,36.95,yellow,credit card,Hudson Sq,Yorkville West,Manhattan,Manhattan
4,2019-03-30 13:27:42,2019-03-30 13:37:14,3,2.16,9.0,1.10,0.0,13.40,yellow,credit card,Midtown East,Yorkville West,Manhattan,Manhattan


In [7]:
# Display data types and info
print("DataFrame Info:")
df_taxi.info()

print("\nBasic Statistics:")
df_taxi.describe()

DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1999 entries, 0 to 1998
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   pickup           1999 non-null   datetime64[ns]
 1   dropoff          1999 non-null   datetime64[ns]
 2   passengers       1999 non-null   int64         
 3   distance         1999 non-null   float64       
 4   fare             1999 non-null   float64       
 5   tip              1999 non-null   float64       
 6   tolls            1999 non-null   float64       
 7   total            1999 non-null   float64       
 8   color            1999 non-null   object        
 9   payment          1981 non-null   object        
 10  pickup_zone      1991 non-null   object        
 11  dropoff_zone     1989 non-null   object        
 12  pickup_borough   1991 non-null   object        
 13  dropoff_borough  1989 non-null   object        
dtypes: datetime64[ns](2), fl

,pickup,dropoff,passengers,distance,fare,tip,tolls,total
count,1999,1999,1999.000000,1999.000000,1999.000000,1999.000000,1999.000000,1999.000000
mean,2019-03-16 09:44:31.230115072,2019-03-16 09:58:28.430715392,1.578789,2.883192,12.655348,2.194247,0.321541,18.618684
min,2019-03-01 00:03:29,2019-03-01 00:13:32,0.000000,0.000000,1.000000,0.000000,0.000000,1.300000
25%,2019-03-08 18:15:23.500000,2019-03-08 18:21:34,1.000000,0.980000,6.500000,0.000000,0.000000,11.300000
50%,2019-03-15 22:59:38,2019-03-15 23:09:31,1.000000,1.600000,9.500000,1.950000,0.000000,14.690000
75%,2019-03-23 18:58:49,2019-03-23 19:08:13,2.000000,3.050000,14.500000,3.000000,0.000000,20.155000
max,2019-03-31 23:43:45,2019-04-01 00:13:58,6.000000,28.300000,120.000000,33.200000,12.500000,166.000000
std,NaN,NaN,1.240245,3.588718,10.541076,2.587149,1.342467,13.266169


In [13]:
# Perform some analytics with pandas
print("Analytics Summary:")
print(f"Average trip distance: {df_taxi['distance'].mean():.2f} miles")
print(f"Average fare amount: ${df_taxi['fare'].mean():.2f}")
print(f"Total revenue: ${df_taxi['total'].sum():.2f}")
print(f"Average tip percentage: {(df_taxi['tip'] / df_taxi['fare'] * 100).mean():.1f}%")

Analytics Summary:
Average trip distance: 2.88 miles
Average fare amount: $12.66
Total revenue: $37218.75
Average tip percentage: 18.3%


In [18]:
# Group by date
df_taxi['pickup'] = pd.to_datetime(df_taxi['pickup']).dt.date

daily_summary = df_taxi.groupby('pickup').agg({
    'distance': 'mean',
    'fare': 'mean',
    'total': 'sum'
}).round(2)

daily_summary.columns = ['avg_distance', 'avg_fare', 'total_revenue']
print("\nDaily Summary:")
daily_summary


Daily Summary:


,avg_distance,avg_fare,total_revenue
pickup,,,
2019-03-01,1.99,9.75,1087.46
2019-03-02,3.21,12.41,1017.66
2019-03-03,3.27,12.45,844.88
2019-03-04,3.33,13.05,1056.20
2019-03-05,2.42,11.14,1243.63
2019-03-06,3.04,13.28,1503.26
2019-03-07,3.11,13.87,1345.31
2019-03-08,2.32,10.87,1473.87
2019-03-09,3.91,15.26,1403.43


In [20]:
# Execute custom SQL query
custom_query = """
SELECT 
    passengers,
    COUNT(*) as num_trips,
    ROUND(AVG(distance), 2) as avg_distance,
    ROUND(AVG(total), 2) as avg_total
FROM raw_taxi_trips
GROUP BY passengers
ORDER BY passengers
"""

df_passenger_analysis = pd.read_sql(custom_query, engine)
print("\nAnalysis by Passenger Count:")
df_passenger_analysis


Analysis by Passenger Count:


,passengers,num_trips,avg_distance,avg_total
0,0,39,3.60,19.30
1,1,1392,2.83,18.38
2,2,316,3.09,19.40
3,3,74,2.37,18.93
4,4,32,2.65,18.50
5,5,94,3.15,19.37
6,6,52,2.81,17.96


### Method 3: Using dbt Commands

```bash
# Show what seeds are available
dbt ls --resource-type seed

# Show compiled SQL for a seed
dbt show --select raw_taxi_trips

# Run a query against the seed using dbt compile
# First create a file: dbt_sql/analyses/check_seed_data.sql
```

Content of `dbt_sql/analyses/check_seed_data.sql`:
```sql
SELECT 
    COUNT(*) as total_rows,
    MIN(pickup_datetime) as earliest_trip,
    MAX(pickup_datetime) as latest_trip,
    AVG(trip_distance) as avg_distance,
    SUM(total_amount) as total_revenue
FROM {{ ref('raw_taxi_trips') }}
```

Then run:
```bash
dbt compile --select check_seed_data
# Check the compiled SQL in target/compiled/dbt_sql/analyses/check_seed_data.sql
```


## Creating dbt Models - The Transformation Layer

Now that we have raw data loaded via seeds, let's transform it using dbt models.

### Model Types and Materialization

dbt supports different materialization strategies:

1. **view** (default): Creates a database view
   - Fast to build
   - Queries run against source data
   - Good for: Light transformations, frequently changing logic

2. **table**: Creates a physical table
   - Slower to build
   - Queries are fast
   - Good for: Heavy transformations, large datasets

3. **incremental**: Appends or updates only new records
   - Most efficient for large datasets
   - Requires unique key
   - Good for: Fact tables, event logs

4. **ephemeral**: Not materialized (inline CTE)
   - Only exists in compiled SQL
   - Good for: Reusable intermediate logic

### Example: Staging Model

Create `dbt_sql/models/staging/stg_taxi_trips.sql`:

```sql
{{ config(
    materialized='view',
    tags=['staging', 'taxi']
) }}

WITH source_data AS (
    SELECT *
    FROM {{ ref('raw_taxi_trips') }}
),

cleaned AS (
    SELECT
        trip_id,
        pickup_datetime,
        dropoff_datetime,
        passenger_count,
        trip_distance,
        fare_amount,
        tip_amount,
        total_amount,
        
        -- Derived columns
        TIMESTAMPDIFF(MINUTE, pickup_datetime, dropoff_datetime) AS trip_duration_minutes,
        DATE(pickup_datetime) AS trip_date,
        HOUR(pickup_datetime) AS pickup_hour,
        DAYNAME(pickup_datetime) AS day_of_week,
        
        -- Calculated metrics
        CASE 
            WHEN trip_distance > 0 
            THEN ROUND(fare_amount / trip_distance, 2)
            ELSE 0 
        END AS fare_per_mile,
        
        CASE 
            WHEN fare_amount > 0 
            THEN ROUND((tip_amount / fare_amount) * 100, 1)
            ELSE 0 
        END AS tip_percentage
        
    FROM source_data
    WHERE trip_distance > 0  -- Data quality filter
        AND fare_amount > 0
        AND pickup_datetime < dropoff_datetime
)

SELECT * FROM cleaned
```

### Running Models

```bash
# Run all models
dbt run

# Run specific model
dbt run --select stg_taxi_trips

# Run models with specific tag
dbt run --select tag:staging

# Run model and all downstream dependencies
dbt run --select stg_taxi_trips+

# Run model and all upstream dependencies
dbt run --select +stg_taxi_trips
```


## Advanced dbt Features

### 1. Testing Data Quality

Create `dbt_sql/models/staging/stg_taxi_trips.yml`:

```yaml
version: 2

models:
  - name: stg_taxi_trips
    description: Cleaned and enriched taxi trip data
    columns:
      - name: trip_id
        description: Unique identifier for each trip
        tests:
          - unique
          - not_null
      
      - name: trip_distance
        description: Trip distance in miles
        tests:
          - not_null
          - dbt_utils.accepted_range:
              min_value: 0
              max_value: 100
      
      - name: passenger_count
        description: Number of passengers
        tests:
          - not_null
          - accepted_values:
              values: [1, 2, 3, 4, 5, 6]
      
      - name: total_amount
        description: Total amount charged
        tests:
          - not_null
          - dbt_utils.expression_is_true:
              expression: ">= fare_amount"
```

Run tests:
```bash
# Run all tests
dbt test

# Test specific model
dbt test --select stg_taxi_trips
```

### 2. Documentation

```bash
# Generate documentation
dbt docs generate

# Serve documentation site
dbt docs serve
```

This creates an interactive website showing:
- Data lineage (DAG visualization)
- Column-level documentation
- Test results
- Source freshness

### 3. Macros - Reusable SQL

Create `dbt_sql/macros/cents_to_dollars.sql`:

```sql
{% macro cents_to_dollars(column_name) %}
    ROUND({{ column_name }} / 100, 2)
{% endmacro %}
```

Use in models:
```sql
SELECT
    trip_id,
    {{ cents_to_dollars('fare_amount') }} AS fare_dollars
FROM {{ ref('raw_taxi_trips') }}
```

### 4. Incremental Models

Create `dbt_sql/models/marts/fct_taxi_trips.sql`:

```sql
{{ config(
    materialized='incremental',
    unique_key='trip_id'
) }}

SELECT
    trip_id,
    pickup_datetime,
    total_amount,
    trip_distance
FROM {{ ref('stg_taxi_trips') }}

{% if is_incremental() %}
    WHERE pickup_datetime > (SELECT MAX(pickup_datetime) FROM {{ this }})
{% endif %}
```

Run incremental:
```bash
# First run (full refresh)
dbt run --select fct_taxi_trips --full-refresh

# Subsequent runs (incremental)
dbt run --select fct_taxi_trips
```


## Complete Workflow Example

Let's put it all together with a complete analytics workflow:

### Directory Structure
```
dbt_sql/models/
├── staging/
│   ├── stg_taxi_trips.sql
│   └── stg_taxi_trips.yml
├── intermediate/
│   └── int_daily_metrics.sql
└── marts/
    ├── fct_trips.sql
    └── dim_date.sql
```

### Step-by-Step Execution

```bash
# 1. Navigate to project
cd DTB-FOR-SQL/dbt_sql

# 2. Load seed data
dbt seed

# 3. Run all models
dbt run

# 4. Test data quality
dbt test

# 5. Generate documentation
dbt docs generate

# 6. View documentation
dbt docs serve
```

### Or run everything at once:

```bash
dbt build
```

This runs: seed → run → test in dependency order.


## Real-World Example: Building a Complete Data Model

Let's create a complete analytics model for taxi revenue analysis.

### Intermediate Model: Daily Aggregations

`dbt_sql/models/intermediate/int_daily_metrics.sql`:

```sql
{{ config(
    materialized='table',
    tags=['intermediate', 'daily']
) }}

WITH daily_trips AS (
    SELECT
        trip_date,
        day_of_week,
        COUNT(*) AS num_trips,
        COUNT(DISTINCT trip_id) AS unique_trips,
        SUM(passenger_count) AS total_passengers,
        
        -- Distance metrics
        ROUND(SUM(trip_distance), 2) AS total_distance,
        ROUND(AVG(trip_distance), 2) AS avg_distance,
        ROUND(MAX(trip_distance), 2) AS max_distance,
        
        -- Revenue metrics
        ROUND(SUM(total_amount), 2) AS total_revenue,
        ROUND(AVG(total_amount), 2) AS avg_revenue_per_trip,
        ROUND(SUM(tip_amount), 2) AS total_tips,
        ROUND(AVG(tip_percentage), 1) AS avg_tip_percentage,
        
        -- Time metrics
        ROUND(AVG(trip_duration_minutes), 1) AS avg_duration_minutes
        
    FROM {{ ref('stg_taxi_trips') }}
    GROUP BY trip_date, day_of_week
)

SELECT
    *,
    -- Calculate revenue per mile
    CASE 
        WHEN total_distance > 0 
        THEN ROUND(total_revenue / total_distance, 2)
        ELSE 0 
    END AS revenue_per_mile
FROM daily_trips
```

### Mart Model: Final Analytics Table

`dbt_sql/models/marts/fct_daily_revenue.sql`:

```sql
{{ config(
    materialized='table',
    tags=['marts', 'revenue']
) }}

SELECT
    trip_date,
    day_of_week,
    num_trips,
    total_revenue,
    avg_revenue_per_trip,
    total_tips,
    
    -- Week-over-week comparison
    LAG(total_revenue, 7) OVER (ORDER BY trip_date) AS revenue_last_week,
    ROUND(
        ((total_revenue - LAG(total_revenue, 7) OVER (ORDER BY trip_date)) 
        / LAG(total_revenue, 7) OVER (ORDER BY trip_date)) * 100,
        1
    ) AS revenue_wow_change_pct,
    
    -- Moving averages
    ROUND(
        AVG(total_revenue) OVER (
            ORDER BY trip_date 
            ROWS BETWEEN 6 PRECEDING AND CURRENT ROW
        ),
        2
    ) AS revenue_7day_ma
    
FROM {{ ref('int_daily_metrics') }}
ORDER BY trip_date
```


## Querying Final Models with Python

After running `dbt run`, query the final marts:


In [ ]:
# Query the final revenue mart
query = """
SELECT 
    trip_date,
    day_of_week,
    num_trips,
    total_revenue,
    revenue_wow_change_pct,
    revenue_7day_ma
FROM fct_daily_revenue
ORDER BY trip_date DESC
LIMIT 10
"""

df_revenue = pd.read_sql(query, engine)
print("Daily Revenue Analysis:")
df_revenue

In [ ]:
# Visualize revenue trends (requires matplotlib)
import matplotlib.pyplot as plt

query_trend = """
SELECT 
    trip_date,
    total_revenue,
    revenue_7day_ma
FROM fct_daily_revenue
ORDER BY trip_date
"""

df_trend = pd.read_sql(query_trend, engine)
df_trend['trip_date'] = pd.to_datetime(df_trend['trip_date'])

plt.figure(figsize=(12, 6))
plt.plot(df_trend['trip_date'], df_trend['total_revenue'], label='Daily Revenue', marker='o')
plt.plot(df_trend['trip_date'], df_trend['revenue_7day_ma'], label='7-Day Moving Average', linewidth=2)
plt.xlabel('Date')
plt.ylabel('Revenue ($)')
plt.title('Taxi Revenue Trends')
plt.legend()
plt.grid(True, alpha=0.3)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## Best Practices and Tips

### 1. Model Organization
- **staging/**: Raw data cleaning, 1:1 with sources
- **intermediate/**: Business logic, joins, aggregations
- **marts/**: Final analytics-ready tables for BI tools

### 2. Naming Conventions
- `stg_` prefix for staging models
- `int_` prefix for intermediate models
- `fct_` prefix for fact tables
- `dim_` prefix for dimension tables

### 3. Performance Optimization
- Use incremental models for large datasets
- Materialize heavy transformations as tables
- Keep staging as views for freshness
- Add indexes on frequently joined columns

### 4. Data Quality
- Always test unique keys and not null constraints
- Use accepted_values for categorical columns
- Implement custom tests for business rules
- Document all assumptions

### 5. Git Workflow
```bash
# Create feature branch
git checkout -b feature/new-revenue-model

# Make changes, test locally
dbt run --select +new_model+
dbt test --select +new_model+

# Commit with clear message
git add .
git commit -m "Add daily revenue analysis model with WoW comparisons"

# Push and create PR
git push origin feature/new-revenue-model
```

### 6. Environment Management
```bash
# Development
dbt run --target dev

# Production
dbt run --target prod
```


## Common dbt Commands Cheat Sheet

```bash
# Setup and Configuration
dbt init project_name              # Initialize new project
dbt debug                          # Test database connection

# Seeds
dbt seed                           # Load all CSV seeds
dbt seed --select seed_name        # Load specific seed
dbt seed --full-refresh            # Drop and recreate seed tables

# Models
dbt run                            # Run all models
dbt run --select model_name        # Run specific model
dbt run --select +model_name       # Run model and upstream dependencies
dbt run --select model_name+       # Run model and downstream dependencies
dbt run --select tag:staging       # Run all models with tag
dbt run --full-refresh             # Full refresh all incremental models

# Testing
dbt test                           # Run all tests
dbt test --select model_name       # Test specific model

# Documentation
dbt docs generate                  # Generate documentation
dbt docs serve                     # Serve documentation site

# Combined Commands
dbt build                          # seed + run + test
dbt build --select model_name      # Build specific model and dependencies

# Utilities
dbt ls                             # List all resources
dbt ls --resource-type model       # List only models
dbt compile                        # Compile SQL without running
dbt clean                          # Delete target/ and dbt_packages/
dbt deps                           # Install packages from packages.yml
```


## Summary

### What We Learned

1. **dbt Fundamentals**
   - dbt transforms data in your warehouse using SQL
   - Brings software engineering practices to analytics
   - Version control, testing, documentation built-in

2. **Seeds**
   - CSV files loaded as database tables
   - Perfect for reference data and small datasets
   - Loaded with `dbt seed`

3. **Models**
   - SQL files that create tables/views
   - Use `{{ ref() }}` for dependencies
   - Different materializations for different needs

4. **Project Structure**
   - staging/ → intermediate/ → marts/
   - Clear separation of concerns
   - Modular and maintainable

5. **Data Quality**
   - Built-in tests (unique, not_null, etc.)
   - Custom tests for business rules
   - Automated testing on every run

### Why This Matters

Traditional analytics:
- SQL scripts scattered everywhere
- No version control
- No testing
- Hard to maintain

With dbt:
- All transformations in Git
- Automated testing
- Self-documenting
- Easy to collaborate
- Production-ready analytics

### Next Steps

1. Practice building models with your own data
2. Explore dbt packages (dbt_utils, dbt_expectations)
3. Set up CI/CD for automated testing
4. Learn about snapshots for historical tracking
5. Integrate with BI tools (Tableau, Power BI, Looker)


## Additional Resources

- **Official dbt Documentation**: https://docs.getdbt.com/
- **dbt Learn**: https://learn.getdbt.com/
- **dbt Discourse**: https://discourse.getdbt.com/
- **dbt Slack Community**: https://www.getdbt.com/community/join-the-community
- **Analytics Engineering Guide**: https://www.getdbt.com/analytics-engineering/
